In [1]:
import pandas as pd


from tqdm import tqdm
import datetime


from AvailableErrors import *
from Andrii.Dir_Task_JAN_FEB.tools_functions import createTimeDifference

In [2]:
global _WEEK_DICT, _HOLIDAY
_WEEK_DICT = {0:'ПН', 1:'ВТ', 2:'СР', 3:'ЧТ', 4:'ПТ', 5:'СБ', 6:'ВС'}
_HOLIDAY_MARKERS = [5,6]

In [4]:
class StrategyExtractor:
    @classmethod
    def ExtractorIndex(cls):
        return 'StrategyExtractor'

    def __init__(self):
        self.compatible = [HubTrendFollowingStrategy.StrategyIndex()]

        self._start = None
        self._end = None
        self.input_data = None

        self.filters = list()
        self.after_filters = list()

        pass

    def input_params(self, start_horizon:datetime.timedelta, end_horizon: datetime.timedelta):
        self._start = start_horizon
        self._end = end_horizon

    def apply_filters(self, single_filter):
        if self.ExtractorIndex not in single_filter.compatible:
            raise StrategyErrors().UnCompatibleFilter(Strategy=self, Filter=single_filter)
        self.filters.append(single_filter)

    def transfer_data(self, data: pd.DataFrame):
        if type(data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.input_data = data

    def making_horizon(self):
        for dot_on_all_data in tqdm(self.input_data.index):
            logical_filter = True
            #   SELECT BORDERS
            buffer_data = self.input_data.loc[dot_on_all_data + self._start: dot_on_all_data + self._end]
            for single_filter in self.filters:
                FILTER = single_filter
                FILTER.apply_buffer_data(buffer_data=buffer_data)
                if not FILTER.condition():
                    logical_filter = False
                    break


            if logical_filter:
                self.after_filters.append(buffer_data)
                break   #   This break scares me

"""================================================================================================================================================="""
class CentralDotFilter:
    def __init__(self):
        self.compatible = [StrategyExtractor.ExtractorIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'CentralDotFilter'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = buffer_data.copy()

    def condition(self, CentralDotCoords):
        pass

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class ConsistingTime:
    def __init__(self,*consist_time):
        self.compatible = [StrategyExtractor.ExtractorIndex]
        self.buffer_data = None
        for _ in consist_time:
            if type(_) != pd._libs.tslibs.timedeltas.Timedelta:
                raise FilterErrors().WrongShiftedType()
        self.consist_time = [_ for _ in consist_time]

    @classmethod
    def FilterIndex(cls):
        return 'ConsistingTime'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        # if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
        #     raise StrategyErrors().WrongIndicesType()
        #
        # ZERO_TIME = buffer_data.index[0].to_numpy()
        # ZERO_TIME = ZERO_TIME.astype('datetime64[s]').item().date()
        # self.buffer_data = buffer_data.index - pd.to_datetime(ZERO_TIME)

        # Если не использовать вызов функции - будет быстрей. Но так зависимости выглядят понятней
        self.buffer_data = createTimeDifference(buffer_data.index)


        return None

    def condition(self):
        for timedelta in self.consist_time:
            #if not timedelta in self.buffer_data.index:
            if not timedelta in self.buffer_data:
                return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class NoHolidays:
    def __init__(self):
        self.compatible = [StrategyExtractor.ExtractorIndex]
        self.buffer_data = None


    @classmethod
    def FilterIndex(cls):
        return 'NoHolidays'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = buffer_data.copy()
        return None

    def condition(self):
        for holiday in self.buffer_data.index.dayofweek:
            if holiday in _HOLIDAY_MARKERS:
                return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""
class LeftBorder:
    def __init__(self, left_border:pd._libs.tslibs.timedeltas.Timedelta):
        self.compatible = [StrategyExtractor.ExtractorIndex]

        if type(left_border) != pd._libs.tslibs.timedeltas.Timedelta:
            raise FilterErrors().WrongShiftedType()
        self.buffer_data = None
        self._left_border = left_border

    @classmethod
    def FilterIndex(cls):
        return 'LeftBorder'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = createTimeDifference(buffer_data.index)
        return None

    def condition(self):
        if self.buffer_data[0] != self._left_border:
            return False

        return True

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""

class RightBorder:
    def __init__(self, right_border:pd._libs.tslibs.timedeltas.Timedelta):
        self.compatible = [StrategyExtractor.ExtractorIndex]

        if type(right_border) != pd._libs.tslibs.timedeltas.Timedelta:
            raise FilterErrors().WrongShiftedType()
        self.buffer_data = None
        self._right_border = right_border

    @classmethod
    def FilterIndex(cls):
        return 'RightBorder'

    def apply_buffer_data(self, buffer_data: pd.DataFrame):
        if type(buffer_data.index) != pd.core.indexes.datetimes.DatetimeIndex:
            raise StrategyErrors().WrongIndicesType()
        self.buffer_data = createTimeDifference(buffer_data.index)
        return None

    def condition(self):
        if self.buffer_data[0] != self._right_border:
            return False

        return True

"""================================================================================================================================================="""

class HubTrendFollowingStrategy:
    @classmethod
    def StrategyIndex(cls):
        return 'HubTrendFollowingStrategy'

    def __init__(self):
        self.all_data = None
        self.central_dot_position = None
        self.lookback = None
        self.potential_hold = None
        self.extractor = None
        self.RulesConstructor = None

    def add_extractor(self, method):
        if self.StrategyIndex() not in method.compatible:
            raise StrategyErrors().UnCompatibleExtractor(HubStrategy=self, StrategyExtractor=method)
        self.extractor = method

    def transfer_data(self, raw_data:pd.DataFrame):
        if type(raw_data) != pd.DataFrame:
            raise TypeError('Input Data must have type pandas DataFrame')
        self.all_data = raw_data.copy()

    def create_lookback_center(self, splitDot_position:int):
        if type(splitDot_position) != int:
            raise SplitDataErrors().WrongCentralDotTypeError()

        self.central_dot_position = splitDot_position

    def testing_simulation(self):
        self.extractor.transfer_data(self.all_data)
        self.extractor.making_horizon()
        return self.extractor.after_filters

    def add_Rules_constructor(self, rulesConstructor):
        if self.StrategyIndex not in rulesConstructor.compatible:
            raise StrategyErrors().UnCompatibleRulesConstructor(HubStrategy=self, RulesConstructor=rulesConstructor)

        self.RulesConstructor = rulesConstructor
"""================================================================================================================================================="""

class TrendFollowingStrategyRulesConstructor:
    def __init__(self):
        self.compatible = [HubTrendFollowingStrategy.StrategyIndex]

        self.opening_rules = list()
        self.holding_rules = list()

        self.data = None
        self.median = None
    @classmethod
    def StrategyRuleConstructorType(cls):
        return 'TrendFollowConstructor'

    def transfer_data(self, tr_data):
        self.data = tr_data.copy()

    def add_median_dot(self, median_dot):
        self.median = median_dot

    def add_opening_rule(self, opening_rule):
        if self.StrategyRuleConstructorType not in opening_rule.compatible:
            raise RulesConstructorErrors().UnCompatibleComplexRule(RuleConstructor=self, ComplexRule=opening_rule)
        self.opening_rules.append(opening_rule)

    def add_hold_rule(self, hold_rule):
        if self.StrategyRuleConstructorType not in hold_rule.compatible:
            raise RulesConstructorErrors().UnCompatibleComplexRule(RuleConstructor=self, ComplexRule=hold_rule)
        self.opening_rules.append(hold_rule)

"""================================================================================================================================================="""
{'OpenChecker': None, 'OperationType': None}

class OpeningCrossHighLowRule:
    def __init__(self):
        self.compatible = [TrendFollowingStrategyRulesConstructor.StrategyRuleConstructorType]

        self.low = None
        self.high = None

    @classmethod
    def RuleIndex(cls):
        return 'OpeningCrossHighLowRule'

    def add_low_calculation(self, low):
        if self.RuleIndex not in low.compatible:
            raise SmallWorkers().UnCompatibleRuleBlock(ComplexRule=self, Worker=low)
        self.low = low

    def add_high_calculation(self, high):
        if self.RuleIndex not in high.compatible:
            raise SmallWorkers().UnCompatibleRuleBlock(ComplexRule=self, Worker=high)
        self.high = high

    def condition(self, current_dot):
        result = {'OpenChecker': False, 'OperationType': None}

        if current_dot > self.high:
            result['OpenChecker'] = True
            result['OperationType'] = 'BUY'

        if current_dot < self.low:
            result["OpenChecker"] = True
            result['OperationType'] = 'SELL'

        return result



"""================================================================================================================================================="""

class CalculateLow:
    def __init__(self):
        self.compatible = [OpeningCrossHighLowRule.RuleIndex]

    @classmethod
    def WorkerIndex(self):
        return 'CalculateLow'

    def testing(self):
        print('LOWER')

"""--------------------------------------------------------------------------------------------------------------------------------------------------"""

class CalculateHigh:
    def __init__(self):
        self.compatible = [OpeningCrossHighLowRule.RuleIndex]

    @classmethod
    def WorkerIndex(self):
        return 'CalculateHigh'
    def testing(self):
        print('HIGHER')
"""================================================================================================================================================="""

"""
horizon = StrategyExtractor()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)
horizon.transfer_data(d.iloc[:1000])
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters
"""

r = OpeningCrossHighLowRule()
r.RuleIndex()
r.add_low_calculation(CalculateLow())
r.add_high_calculation(CalculateHigh())

In [ ]:
d = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
d.index = pd.to_datetime(d.index)
d = d.resample('1T').first()


In [ ]:
horizon = StrategyExtractor()

constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)

nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)

leftBorder = LeftBorder(pd.Timedelta(hours=15))
horizon.apply_filters(leftBorder)

horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))

In [ ]:
TF = HubTrendFollowingStrategy()
TF.add_extractor(horizon)
TF.extractor.after_filters.clear()
TF.transfer_data(d)
TF.testing_simulation()

In [ ]:
horizon = StrategyExtractor()

constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)

nodolidays = NoHolidays()
horizon.apply_filters(nodolidays)

leftBorder = LeftBorder(pd.Timedelta(hours=15))
horizon.apply_filters(leftBorder)

horizon.transfer_data(d)

horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
#horizon.after_filters

TF = HubTrendFollowingStrategy()
TF.extract_data_method(horizon)

In [ ]:
horizon.after_filters

In [ ]:
"""d = pd.read_csv('zip_data/AUDCAD.csv', index_col=0)
d.index = pd.to_datetime(d.index)
d = d.resample('1H').first()
d = d.iloc[:150_0]
horizon = StrategyExtractor()
constime = ConsistingTime(pd.Timedelta(days=0, hours=22), pd.Timedelta(days=1, hours=8), pd.Timedelta(days=1, hours=22))
horizon.apply_filters(constime)
#nodolidays = NoHolidays()
#horizon.apply_filters(nodolidays)
horizon.transfer_data(d)
horizon.input_params(start_horizon=datetime.timedelta(hours=-12), end_horizon=datetime.timedelta(hours=19))
horizon._making_horizon()
horizon.after_filters"""